# **SINALIZADOR DE EXTRATO**.

---


In [6]:
!pip install PyMuPDF

In [ ]:
# @title INSIRA OS EXTRATOS EM PDF.
import os
import fitz
from pathlib import Path
import re
from google.colab import files

PALAVRAS_CHAVE_CC = ["Extrato de Conta Corrente", "Extrato Consolidado"]
PALAVRAS_CHAVE_APLICACAO = ["DEMONSTRATIVO DE EVOLUÇÃO"]

LANÇAMENTOS_PADRAO_CC = {
    "Desbloqueio Folha de Pagamento": {"categoria": "RECEITA"},
    "Tarifa Antecip Float Folha Pagamento": {"categoria": "TARIFA"},
    "Tarifa Pix Enviado": {"categoria": "TARIFA"},
    "Tarifa Folha de Pagamento": {"categoria": "TARIFA"},
    "Bloqueio Folha Pagamento": {"categoria": "DESPESA"},
    "TED Recebida": {"categoria": "RECEITA"},
    "DOC/TED": {"categoria": "TARIFA"},
    "Depósito 101": {"categoria": "RECEITA"},
    "Cheque": {"categoria": "DESPESA"},
    "TARIFA": {"categoria": "TARIFA"},
    "TED Devolvida Mot": {"categoria": "DESPESA"},
    "TED Devolvida Mot": {"categoria": "RECEITA"},
    "Transferência Recebida": {"categoria": "RECEITA"},
    "Transferência Realizada": {"categoria": "DESPESA"},

}

MAPA_DE_PARES = {
    'APLICACAO': 'Aplicação Título Banestes',
    'RESGATE': 'Resgate Título Banestes',
    'RENDIMENTO': None,
}

def extrair_dia_do_nome(nome_arquivo):
    match = re.search(r'_(\d{2})_\d{2}_\d{4}', nome_arquivo)
    if match: return match.group(1)
    return "SEM_DATA"

def criar_pastas_saida(dia_base):
    for subpasta in ["DESPESA", "RECEITA", "TARIFA"]:
        caminho = Path(dia_base) / subpasta
        caminho.mkdir(parents=True, exist_ok=True)

def identificar_tipo_extrato(caminho):
    try:
        with fitz.open(caminho) as doc:
            texto_pagina_um = " ".join(doc[0].get_text().split()).lower()
        if any(" ".join(k.split()).lower() in texto_pagina_um for k in PALAVRAS_CHAVE_APLICACAO): return "APLICACAO"
        if any(" ".join(k.split()).lower() in texto_pagina_um for k in PALAVRAS_CHAVE_CC): return "CORRENTE"
    except Exception as e:
        print(f"[AVISO] Não foi possível ler o arquivo {caminho}: {e}")
    return "INDEFINIDO"

def extrair_parte_comum_nome(nome):
    match = re.search(r'(\d+_\d{2}_\d{2}_\d{4})', nome)
    return match.group(1) if match else None

def encontrar_par_cc(arq_app, lista_cc):
    parte_comum = extrair_parte_comum_nome(arq_app)
    if not parte_comum: return None
    return next((arq for arq in lista_cc if parte_comum in arq), None)

def analisar_transacoes_por_dia(page):
    transacoes_por_dia = {}
    COLUNAS = {
        "DATA": (20, 90), "RENDIMENTO": (95, 170),
        "APLICACAO": (240, 320), "RESGATE LIQUIDO": (320, 420),
    }
    words = page.get_text("words")
    if not words: return {}
    lines = {}
    for w in words:
        y_key = int(w[1])
        if y_key not in lines: lines[y_key] = []
        lines[y_key].append(w)

    for y_key in sorted(lines.keys()):
        line_words = sorted(lines[y_key], key=lambda w: w[0])
        line_text = " ".join(w[4] for w in line_words)
        match_data = re.match(r'^\d{1,2}/\d{2}/\d{4}', line_text)
        if match_data:
            data_str = match_data.group(0)
            dia = data_str.split('/')[0].zfill(2)
            if dia not in transacoes_por_dia: transacoes_por_dia[dia] = []
            data_word = next((w for w in line_words if w[4] == data_str), None)
            for nome_coluna, (x0, x1) in COLUNAS.items():
                if nome_coluna == "DATA": continue
                for word in line_words:
                    if x0 < word[0] < x1:
                        try:
                            if float(word[4].replace('.', '').replace(',', '.')) > 0:
                                transacoes_por_dia[dia].append({
                                    'tipo': nome_coluna,
                                    'data_rect': fitz.Rect(data_word[:4]) if data_word else None,
                                    'valor_rect': fitz.Rect(word[:4])
                                })
                                break
                        except ValueError: continue
    return transacoes_por_dia

def highlight_transacao_aplicacao(page, transacao):
    if transacao:
        if transacao.get('data_rect'): page.add_highlight_annot(transacao['data_rect'])
        if transacao.get('valor_rect'): page.add_highlight_annot(transacao['valor_rect'])

def executar_par(arq_app, arq_cc, dia_base, tipo_evento, transacao):
    print(f"  -> Evento '{tipo_evento}' no dia {dia_base}. Processando...")
    regras = {
        'APLICACAO': {'pasta_app': 'RECEITA', 'titulo_app': '_APLICACAO', 'pasta_cc': 'DESPESA', 'titulo_cc': '_TRANSF_CONTA_APLICACAO'},
        'RESGATE LIQUIDO': {'pasta_app': 'DESPESA', 'titulo_app': '_RESGATE', 'pasta_cc': 'RECEITA', 'titulo_cc': '_TRANSF_CONTA_MOVIMENTO'},
        'RENDIMENTO': {'pasta_app': 'RECEITA', 'titulo_app': '_RENDIMENTO', 'pasta_cc': None, 'titulo_cc': None}
    }
    regra_atual = regras.get(tipo_evento)
    if not regra_atual: return

    try:
        with fitz.open(arq_app) as doc:
            highlight_transacao_aplicacao(doc[0], transacao)
            novo_nome = f"{Path(arq_app).stem}{regra_atual['titulo_app']}.pdf".upper()
            caminho_saida = Path(dia_base) / regra_atual['pasta_app'] / novo_nome
            doc.save(caminho_saida, garbage=4, deflate=True)
            print(f"     -> Extrato Aplicação salvo em: {caminho_saida}")
    except Exception as e: print(f"     [ERRO] ao processar '{arq_app}': {e}")

    if not regra_atual.get('pasta_cc'):
        print("     -> Evento sem par na C/C. Processamento do par concluído.")
        return

    try:
        with fitz.open(arq_cc) as doc:
            modificado = False
            if tipo_evento == 'RESGATE LIQUIDO' or tipo_evento == 'APLICACAO':
                print(f"     -> Regra de Par: buscando número da conta para evento '{tipo_evento}'.")
                for page in doc:
                    words = page.get_text("words")
                    for i, w in enumerate(words):
                        if w[4] == "Conta:" and i + 1 < len(words):
                            account_word = words[i+1]
                            if abs(w[1] - account_word[1]) < 5:
                                page.add_highlight_annot(fitz.Rect(account_word[:4]))
                                modificado = True
                                break
                    if modificado: break

            if modificado:
                novo_nome = f"{Path(arq_cc).stem}{regra_atual['titulo_cc']}.pdf".upper()
                caminho_saida = Path(dia_base) / regra_atual['pasta_cc'] / novo_nome
                doc.save(caminho_saida, garbage=4, deflate=True)
                print(f"     -> Extrato C/C salvo em: {caminho_saida}")
            else:
                 print(f"     -> Nenhuma modificação necessária ou possível no par C/C para '{tipo_evento}'.")

    except Exception as e: print(f"     [ERRO] ao processar '{arq_cc}': {e}")


def processar_lancamentos_padrao_cc(caminho_arquivo, dia_base, start_counter):
    print(f"  Processando lançamentos padrão de: {Path(caminho_arquivo).name}")
    lancamentos_encontrados = []

    termos_ordenados = sorted(LANÇAMENTOS_PADRAO_CC.keys(), key=len, reverse=True)

    try:
        with fitz.open(caminho_arquivo) as doc:
            for page in doc:
                blocks = page.get_text("blocks")
                for b in blocks:
                    line_text = b[4].replace('\n', ' ')
                    for termo_busca in termos_ordenados:
                        if termo_busca in line_text:
                            lancamentos_encontrados.append({
                                "termo": termo_busca,
                                "config": LANÇAMENTOS_PADRAO_CC[termo_busca],
                                "inst": fitz.Rect(b[:4]),
                                "page_num": page.number
                            })
                            break
    except Exception as e:
        print(f"     [ERRO] ao ler o arquivo C/C '{caminho_arquivo}': {e}")
        return start_counter

    tarifa_counter = start_counter
    lancamento_counts = {}
    for lancamento in lancamentos_encontrados:
        try:
            with fitz.open(caminho_arquivo) as doc_escrita:
                pag = doc_escrita[lancamento["page_num"]]
                pag.add_highlight_annot(lancamento["inst"])

                categoria = lancamento["config"]["categoria"]
                termo_busca = lancamento["termo"]
                base_nome_arquivo = Path(caminho_arquivo).stem

                if categoria == "TARIFA":
                    titulo = f"TARIFA_{tarifa_counter:02d}"
                    tarifa_counter += 1
                    novo_nome = f"{base_nome_arquivo}_{titulo}.pdf".upper()
                else:
                    titulo = termo_busca.replace(" ", "_").replace("/", "_")
                    lancamento_counts[titulo] = lancamento_counts.get(titulo, 0) + 1
                    count = lancamento_counts[titulo]
                    if count > 1:
                        novo_nome = f"{base_nome_arquivo}_{titulo}_{count:02d}.pdf".upper()
                    else:
                        novo_nome = f"{base_nome_arquivo}_{titulo}.pdf".upper()

                caminho_saida = Path(dia_base) / categoria / novo_nome
                doc_escrita.save(caminho_saida, garbage=4, deflate=True)
                print(f"     -> Lançamento '{termo_busca}' salvo em: {caminho_saida}")

        except Exception as e:
            print(f"     [ERRO] ao salvar o arquivo para o lançamento '{lancamento['termo']}': {e}")

    return tarifa_counter

def processar_transferencias_entre_contas(extratos_cc):
    for arq_saida in extratos_cc:
        dia_base = extrair_dia_do_nome(Path(arq_saida).name)
        ocorrencias = []
        try:
            with fitz.open(arq_saida) as doc:
                for page_num, page in enumerate(doc):
                    blocks = page.get_text("blocks")
                    for b in blocks:
                        line_text = b[4]
                        if "Transferência Realizada" in line_text and "INSTITUTO ACQUA" in line_text:
                            ocorrencias.append({"page_num": page_num, "rect": fitz.Rect(b[:4])})

            for item in ocorrencias:
                with fitz.open(arq_saida) as doc_escrita:
                    pag = doc_escrita[item["page_num"]]
                    pag.add_highlight_annot(item["rect"])
                    novo_nome = f"{Path(arq_saida).stem}_TRANSF_ENTRE_CONTAS.pdf".upper()
                    caminho_saida = Path(dia_base) / "DESPESA" / novo_nome
                    doc_escrita.save(caminho_saida, garbage=4, deflate=True)
                    print(f"     -> Transferência entre contas (Saída) salva em: {caminho_saida}")
        except Exception as e:
            print(f"     [ERRO] ao processar Transferência Realizada em '{arq_saida}': {e}")

    for arq_entrada in extratos_cc:
        dia_base = extrair_dia_do_nome(Path(arq_entrada).name)
        try:
            with fitz.open(arq_entrada) as doc:
                for page in doc:
                    for inst in page.search_for("Transferência Recebida"):
                        with fitz.open(arq_entrada) as doc_escrita:
                            pag = doc_escrita[page.number]
                            pag.add_highlight_annot(fitz.Rect(0, inst.y0, pag.rect.width, inst.y1))
                            novo_nome = f"{Path(arq_entrada).stem}_TRANSFERENCIA_RECEBIDA.pdf".upper()
                            caminho_saida = Path(dia_base) / "RECEITA" / novo_nome
                            doc_escrita.save(caminho_saida, garbage=4, deflate=True)
                            print(f"     -> Transferência entre contas (Entrada) salva em: {caminho_saida}")
        except Exception as e:
            print(f"     [ERRO] ao processar Transferência Recebida em '{arq_entrada}': {e}")


def main():
    print("Por favor, faça o upload dos extratos.")
    try:
        uploaded = files.upload()
    except Exception: print("Não foi possível fazer o upload. Execute em um ambiente Google Colab."); return

    if not uploaded: print("Nenhum arquivo enviado."); return

    arquivos = list(uploaded.keys())
    extratos_cc, extratos_app = [], []

    print("\n--- Fase 1: Identificando extratos ---")
    for arq in arquivos:
        tipo = identificar_tipo_extrato(arq)
        if tipo == "CORRENTE": extratos_cc.append(arq)
        elif tipo == "APLICACAO": extratos_app.append(arq)
        print(f"  -> {arq} ... {tipo}")
    print("-" * 40)

    print("\n--- Fase 2: Processando Pares (Aplicação x C/C) ---")
    if extratos_app:
        for arq_app in extratos_app:
            par_cc = encontrar_par_cc(arq_app, extratos_cc)
            if not par_cc: print(f"  [AVISO] '{arq_app}' não encontrou um par de C/C."); continue

            dia_do_par = extrair_dia_do_nome(Path(par_cc).name)
            criar_pastas_saida(dia_do_par)

            with fitz.open(arq_app) as doc_app:
                transacoes_do_mes = analisar_transacoes_por_dia(doc_app[0])

            if dia_do_par in transacoes_do_mes:
                transacoes_do_dia = transacoes_do_mes[dia_do_par]
                for transacao in transacoes_do_dia:
                    executar_par(arq_app, par_cc, dia_do_par, transacao['tipo'], transacao)
            else:
                print(f"  Nenhuma transação de aplicação/resgate/rendimento encontrada para o dia {dia_do_par} no arquivo {Path(arq_app).name}")
    print("-" * 40)

    if len(extratos_cc) > 1:
        print("\n--- Fase 3: Processando Transferências Entre Contas ---")
        processar_transferencias_entre_contas(extratos_cc)
        print("-" * 40)

    print("\n--- Fase 4: Processando Lançamentos Padrão da C/C ---")
    tarifa_counter = 1
    for arq_cc in extratos_cc:
        dia = extrair_dia_do_nome(Path(arq_cc).name)
        criar_pastas_saida(dia)
        tarifa_counter = processar_lancamentos_padrao_cc(arq_cc, dia, tarifa_counter)
    print("-" * 40)

    print("\nProcessamento concluído com sucesso!")

if __name__ == "__main__":
    main()

Por favor, faça o upload dos extratos.


# **ZIPANDO AS PASTAS  E BAIXANDO**

In [ ]:
# @title Clique no botão para que os extratos sinalizados fiquem disponíveis para download de forma unificada.
import os
import shutil
import re

PASTA_RAIZ = 'Extratos_Organizados'
NOME_ARQUIVO_ZIP = 'EXTRATOS SINALIZADOS'
NOME_ARQUIVO_ZIP_COM_EXTENSAO = f'{NOME_ARQUIVO_ZIP}.zip'

print("Limpando resultados antigos...")
if os.path.exists(PASTA_RAIZ):
    shutil.rmtree(PASTA_RAIZ)
    print(f"Pasta antiga '{PASTA_RAIZ}' removida.")

if os.path.exists(NOME_ARQUIVO_ZIP_COM_EXTENSAO):
    os.remove(NOME_ARQUIVO_ZIP_COM_EXTENSAO)
    print(f"Arquivo '{NOME_ARQUIVO_ZIP_COM_EXTENSAO}' antigo removido.")
print("-" * 20)

os.makedirs(PASTA_RAIZ)

todos_os_itens = os.listdir('.')
pastas_movidas = False

for item in todos_os_itens:
    if os.path.isdir(item) and (re.fullmatch(r'\d+', item) or item == 'SEM_DATA'):
        shutil.move(item, PASTA_RAIZ)
        print(f"Movendo pasta '{item}' para dentro de '{PASTA_RAIZ}/'")
        pastas_movidas = True

if pastas_movidas:
    print("\nCompactando tudo em um único arquivo .zip...")
    shutil.make_archive(NOME_ARQUIVO_ZIP, 'zip', PASTA_RAIZ)
    print(f"\nConcluído! O arquivo '{NOME_ARQUIVO_ZIP_COM_EXTENSAO}' foi criado.")
    print("Agora você pode baixá-lo manualmente pelo painel de arquivos à esquerda.")
else:
    print("\nNenhuma pasta de resultado encontrada para compactar.")

Limpando resultados antigos...
--------------------
Movendo pasta '11' para dentro de 'Extratos_Organizados/'

Compactando tudo em um único arquivo .zip...

Concluído! O arquivo 'EXTRATOS SINALIZADOS.zip' foi criado.
Agora você pode baixá-lo manualmente pelo painel de arquivos à esquerda.


# **LIMPANDO O PROGRAMA**

In [ ]:
# @title Clique no botão para que os arquivos sejam deletados para o próximo D+1.
import os
import shutil
import re

print("--- INICIANDO LIMPEZA COMPLETA DO AMBIENTE ---")

# Lista de alvos com nomes fixos para remoção (agora inclui o novo arquivo zip)
alvos_fixos = [
    'Extratos_Organizados',
    'Extratos_Processados.zip',
    'EXTRATOS SINALIZADOS.ZIP'
]
itens_removidos = 0

# Pega todos os itens no diretório atual
try:
    itens_no_diretorio = os.listdir('.')
except Exception as e:
    print(f"❌ Não foi possível listar os arquivos no diretório. Erro: {e}")
    itens_no_diretorio = []

# Cria uma lista com os alvos em minúsculas para comparação
alvos_fixos_lower = [alvo.lower() for alvo in alvos_fixos]

for item in itens_no_diretorio:
    # Condição 1: Apaga pastas de resultado (ex: '30' ou 'SEM_DATA')
    if os.path.isdir(item) and (item.isdigit() or item == 'SEM_DATA'):
        try:
            shutil.rmtree(item)
            print(f"Pasta de resultados '{item}/' removida.")
            itens_removidos += 1
        except Exception as e:
            print(f"Erro ao remover a pasta '{item}': {e}")

    # Condição 2: Apaga a pasta de organização do ZIP e os próprios ZIPs (agora case-insensitive)
    elif item.lower() in alvos_fixos_lower:
        try:
            if os.path.isdir(item):
                shutil.rmtree(item)
                print(f"Pasta de organização '{item}/' removida.")
            elif os.path.isfile(item):
                os.remove(item)
                print(f"Arquivo ZIP '{item}' removido.")
            itens_removidos += 1
        except Exception as e:
            print(f"Erro ao remover '{item}': {e}")

    # Condição 3: Apaga todos os arquivos .pdf originais
    elif os.path.isfile(item) and item.lower().endswith('.pdf'):
        try:
            os.remove(item)
            itens_removidos += 1
        except Exception as e:
            print(f"Erro ao remover o arquivo PDF '{item}': {e}")

print("-" * 20)
if itens_removidos == 0:
    print("Nenhum item para limpar foi encontrado.")
else:
    print("Limpeza completa concluída com sucesso!")

--- INICIANDO LIMPEZA COMPLETA DO AMBIENTE ---
Pasta de organização 'Extratos_Organizados/' removida.
Arquivo ZIP 'EXTRATOS SINALIZADOS.zip' removido.
--------------------
Limpeza completa concluída com sucesso!
